# Crime since 2001 in Chicago

* Data source: [data.gov](https://catalog.data.gov/dataset/crimes-2001-to-present-398a4)


* Describe the dataset:

    This dataset reflects reported incidents of crime that occurred in the City of Chicago from 2001 to August 9, 2018, minus the most recent seven days.

    Columns in the dataset: ['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location']


* Outline the types of insights you hope to gain from it:

    1. Most common type of crime each year and each date.
    1. Tracking crime frequency by geohash(7) per year.
    1. 


* Make hypotheses about what you might find:

    1. A lot of crimes that are categorized as [Battery](https://en.wikipedia.org/wiki/Battery_(crime) type during Black Friday.
    1. Most of the crimes happened in [these neighborhoods](https://www.roadsnacks.net/these-are-the-10-worst-chicago-neighborhoods/).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
import time

In [25]:
%%time

df = spark.read.format('csv')\
    .option('header', 'true')\
    .load('hdfs://orion11:13030/crime-since-2001-chicago.csv')\
    .limit(100)
df.show(5)

+--------+-----------+--------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR| Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10000092|   HY189866|03/18/2015 07:44:...|     047XX W OHIO ST|041A|      BATTERY| AGGRAVATED: HANDGUN|              STREET| false|   false|1111

In [3]:
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

## Most common type of crime each year and each date.

In [21]:
serialize_date = udf(lambda d: datetime.strptime(d, '%m/%d/%Y %I:%M:%S %p'), DateType())

In [26]:
%%time

df = df.withColumn('Date', serialize_date(df['Date']))

df.show(3)

+-------+-----------+----------+-----------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|      Date|            Block|IUCR| Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+----------+-----------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|5833185|   HN640275|2007-10-10|038XX W OGDEN AVE|502P|OTHER OFFENSE|FALSE/STOLEN/ALTE...|              STREET| false|   false|1014|     010|  24|            29|      26|     1151189|    

In [ ]:
%%time

df.createOrReplaceTempView('crime')
most = spark.sql(
    '''
    SELECT
        Month,
        AVG(Humidity) AS Humidity,
        AVG(Temperature) AS Temperature,
        AVG(Wind) AS Wind
    FROM crime
    WHERE Geohash like "9q8y%"
    GROUP BY Month
    '''
)

most.show()